# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [32]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [55]:
city_data_to_load = "../WeatherPy/output_data/cities.csv"

# Read city data into dataframe
city_data = pd.read_csv(city_data_to_load)
city_data.head()


,Unnamed: 0,city,lat,lon,MaxTemp,Humidity,Cloudiness,WindSpeed,Country,Date
0,0,Saint-Philippe,-21.36,55.77,78.80,78,0,18.34,RE,1608484290
1,1,Sinnamary,5.38,-52.95,82.92,75,84,7.76,GF,1608484290
2,2,Atuona,-9.80,-139.03,78.37,77,0,19.35,PF,1608484290
3,3,Kaeo,-35.10,173.78,59.00,96,43,3.00,NZ,1608484291
4,4,Hobyo,5.35,48.53,79.12,76,0,19.66,SO,1608484291


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [33]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [52]:
city_data = city_data.dropna()
# Store 'lat' and 'lon' into  locations 
locations = city_data[["lat", "lon"]] #.astype(float)
#locations.count()
# Convert Humidity to float and store
# HINT: be sure to handle NaN values

humidity = city_data["Humidity"].astype(float)




In [54]:
# Create a Humidityy Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)
#heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
#                                 dissipating=False, max_intensity=100,
#                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [61]:
hotel_df = city_data[(city_data["MaxTemp"] >= 70) &
                       (city_data["MaxTemp"] <= 80) &
                       (city_data["Cloudiness"] == 0) &
                       (city_data["Humidity"] < 80)]
hotel_df.head(17)

,Unnamed: 0,city,lat,lon,MaxTemp,Humidity,Cloudiness,WindSpeed,Country,Date
0,0,Saint-Philippe,-21.36,55.77,78.80,78,0,18.34,RE,1608484290
2,2,Atuona,-9.80,-139.03,78.37,77,0,19.35,PF,1608484290
4,4,Hobyo,5.35,48.53,79.12,76,0,19.66,SO,1608484291
19,19,Yumbel,-37.08,-72.57,71.60,40,0,12.75,CL,1608484293
26,26,Carnarvon,-24.87,113.63,77.00,73,0,25.28,AU,1608483995
47,47,Makkah al Mukarramah,21.43,39.83,76.75,57,0,3.87,SA,1608484184
58,58,Umluj,25.02,37.27,75.56,60,0,4.54,SA,1608484299
81,81,Chui,-33.70,-53.46,74.62,43,0,15.90,UY,1608484303
88,88,Tanout,14.97,8.89,76.87,17,0,9.69,NE,1608484229
94,94,Tessalit,20.20,1.01,74.14,19,0,10.85,ML,1608484304


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [67]:

hotel_df['HotelName'] = ""
hotel_df

/opt/anaconda3/envs/case_python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,city,lat,lon,MaxTemp,Humidity,Cloudiness,WindSpeed,Country,Date,HotelName
0,0,Saint-Philippe,-21.36,55.77,78.80,78,0,18.34,RE,1608484290,
2,2,Atuona,-9.80,-139.03,78.37,77,0,19.35,PF,1608484290,
4,4,Hobyo,5.35,48.53,79.12,76,0,19.66,SO,1608484291,
19,19,Yumbel,-37.08,-72.57,71.60,40,0,12.75,CL,1608484293,
26,26,Carnarvon,-24.87,113.63,77.00,73,0,25.28,AU,1608483995,
47,47,Makkah al Mukarramah,21.43,39.83,76.75,57,0,3.87,SA,1608484184,
58,58,Umluj,25.02,37.27,75.56,60,0,4.54,SA,1608484299,
81,81,Chui,-33.70,-53.46,74.62,43,0,15.90,UY,1608484303,
88,88,Tanout,14.97,8.89,76.87,17,0,9.69,NE,1608484229,
94,94,Tessalit,20.20,1.01,74.14,19,0,10.85,ML,1608484304,


In [ ]:
# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "39.952583,-75.16522",  # philadelphia coords
    "rankby": "distance",
    "type": "restaurant",
    "key": gkey,
}
# use iterrows to iterate through pandas dataframe
for index, row in types_df.iterrows():

    # get restaurant type from df
    restr_type = row['ethnicity']

    # add keyword to params dict
    params['keyword'] = restr_type

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {restr_type}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
        types_df.loc[index, 'name'] = results[0]['name']
        types_df.loc[index, 'address'] = results[0]['vicinity']
        types_df.loc[index, 'price_level'] = results[0]['price_level']
        types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
